# Deliverable 1. Retrieve Weather Data.
---
1. Create a folder called `Weather_Database` to save all the files related with this deliverable.

2. Save the `Weather_Database_starter_code.ipynb` starter code to the `Weather_Database` folder and rename it as `Weather_Database.ipynb`.

3. Use the `np.random.uniform` function to generate a new set of 2,000 random latitudes and 2,000 longitudes.

4. Use the `citipy` module to get the nearest city for each latitude and longitude combination.

5. Import your OpenWeatherMap's API key and assemble the API call URL as a string variable. Recall to edit the `config.py` file to add your API key; also, it's critical to avoid publishing your API key on your GitHub repository.

6. Retrieve the following information from the API call:

    * Latitude and longitude

    * Maximum temperature

    * Percent humidity

    * Percent cloudiness

    * Wind speed

    * Weather description (for example, clouds, fog, light rain, clear sky)

7. Add the weather data to a new DataFrame.

8. Export the DataFrame as a CSV file, and save it as `WeatherPy_Database.csv` in the `Weather_Database` folder.
---

In [1]:
import numpy as np
import pandas as pd
from citipy import citipy
import requests
import time
from datetime import datetime
from config import weather_api_key

In [2]:
NUM_POINTS = 2000    # DEBUG 2000
BASE_URL = ("https://api.openweathermap.org/data/2.5/weather?"
            f"appid={weather_api_key}&units=imperial")
OUTPUT_DATA_PATH = "cities.csv"

## Use the `np.random.uniform` function to generate a new set of 2,000 random latitudes and 2,000 longitudes.

In [3]:
# Create a set of random latitude and longitude combinations
lats = np.random.uniform(low=-90.000, high=90.000, size=NUM_POINTS)
lngs = np.random.uniform(low=-180.000, high=180.000, size=NUM_POINTS)

# Use the zip function to create an iterator of tuples containing the latitude and longitude combinations
lat_lngs = zip(lats, lngs)
lat_lngs

# Add the latitudes and longitudes to a list
coordinates = list(lat_lngs)

In [4]:

# for coordinate in coordinates:
#     print(coordinate[0], coordinate[1])
print(len(coordinates))
print(coordinates[0])
print(coordinates[1])

2000
(46.78593325612607, -82.0231668276165)
(22.529704430030975, 23.94690671709705)


## Use the `citipy` module to get the nearest city for each latitude and longitude combination.

In [5]:

# Create an empty list for holding the cities
cities = []

# Use a for loop to identify nearest city for each latitude and longitude combination using the citipy module
for coordinate in coordinates:
    # Use the citipy module to identify the nearest city to the current coordinate
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
    city
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
print(len(cities))
print(cities)


775
['espanola', 'faya', 'xuanzhou', 'san policarpo', 'puerto ayora', 'belushya guba', 'attawapiskat', 'rikitea', 'fairbanks', 'east london', 'avarua', 'kamenka', 'butaritari', 'halalo', 'tuktoyaktuk', 'coquimbo', 'bredasdorp', 'thompson', 'praia', 'the valley', 'yulara', 'mataura', 'yaan', 'binzhou', 'waipawa', 'kavieng', 'iqaluit', 'mahina', 'buluang', 'tasiilaq', 'busselton', 'khatanga', 'lithakia', 'hermanus', 'hobart', 'kloulklubed', 'pitimbu', 'bilma', 'saposoa', 'port alfred', 'atuona', 'beloha', 'narsaq', 'kapaa', 'guangzhou', 'ancud', 'raga', 'laguna', 'nocera superiore', 'tanout', 'taolanaro', 'ostrovnoy', 'takoradi', 'lavrentiya', 'bilibino', 'chokurdakh', 'tabulbah', 'ilulissat', 'mar del plata', 'guaymas', 'santiago del estero', 'castro', 'bam', 'nanakuli', 'solovetskiy', 'vaini', 'mys shmidta', 'barrow', 'praia da vitoria', 'kologriv', 'palmer', 'katsuura', 'nikolskoye', 'zyryanka', 'albany', 'ushuaia', 'clyde river', 'hualmay', 'kudahuvadhoo', 'saint-philippe', 'whitehor

## Import your OpenWeatherMap's API key and assemble the API call URL as a string variable. Recall to edit the `config.py` file to add your API key.

## Retrieve the following information from the API call.

- The latitude and longitude
- The Max temperature
- The % humidity
- The % cloudiness
- The Wind speed
- The Weather description, i.e., cloudy, fog, light rain, clear sky, etc. 

In [6]:
# Create an empty list to hold weather data for each city
city_data = []

# Print a message to indicate that the data retrieval starts
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters and set them to 1
record_count = 1
set_count = 1

# Loop through all the cities in our list to fetch weather data for each city
for i, city in enumerate(cities):
        
    # Group cities in sets of 50 for logging purposes
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create an endpoint URL for each city
    city_url = BASE_URL + "&q=" + city.replace(" ","+")
    
    # Log the url, record, and set numbers
    print(f"Processing Record {record_count} of Set {set_count} | {city}")

    # Add 1 to the record count
    record_count += 1

    # Run an API request for each of the cities
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]                
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date})

# If an error is experienced, skip the city.
    except KeyError:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | espanola
Processing Record 2 of Set 1 | faya
Processing Record 3 of Set 1 | xuanzhou
Processing Record 4 of Set 1 | san policarpo
Processing Record 5 of Set 1 | puerto ayora
Processing Record 6 of Set 1 | belushya guba
City not found. Skipping...
Processing Record 7 of Set 1 | attawapiskat
City not found. Skipping...
Processing Record 8 of Set 1 | rikitea
Processing Record 9 of Set 1 | fairbanks
Processing Record 10 of Set 1 | east london
Processing Record 11 of Set 1 | avarua
Processing Record 12 of Set 1 | kamenka
Processing Record 13 of Set 1 | butaritari
Processing Record 14 of Set 1 | halalo
City not found. Skipping...
Processing Record 15 of Set 1 | tuktoyaktuk
Processing Record 16 of Set 1 | coquimbo
Processing Record 17 of Set 1 | bredasdorp
Processing Record 18 of Set 1 | thompson
Processing Record 19 of Set 1 | praia
Processing Record 20 of Set 1 | the valley
Processing Record 21 of Set

Processing Record 40 of Set 4 | namatanai
Processing Record 41 of Set 4 | hasaki
Processing Record 42 of Set 4 | mahibadhoo
Processing Record 43 of Set 4 | shimoda
Processing Record 44 of Set 4 | clinton
Processing Record 45 of Set 4 | ampanihy
Processing Record 46 of Set 4 | rio gallegos
Processing Record 47 of Set 4 | belmonte
Processing Record 48 of Set 4 | chitral
Processing Record 49 of Set 4 | saint-marc-des-carrieres
Processing Record 50 of Set 4 | jacareacanga
Processing Record 1 of Set 5 | zelenoborskiy
Processing Record 2 of Set 5 | lorengau
Processing Record 3 of Set 5 | byron bay
Processing Record 4 of Set 5 | ambon
Processing Record 5 of Set 5 | halifax
Processing Record 6 of Set 5 | deputatskiy
Processing Record 7 of Set 5 | tambura
Processing Record 8 of Set 5 | lebu
Processing Record 9 of Set 5 | khorixas
Processing Record 10 of Set 5 | ler
Processing Record 11 of Set 5 | balikpapan
Processing Record 12 of Set 5 | dikson
Processing Record 13 of Set 5 | kaitangata
Proces

Processing Record 27 of Set 8 | barawe
City not found. Skipping...
Processing Record 28 of Set 8 | cururupu
Processing Record 29 of Set 8 | cherskiy
Processing Record 30 of Set 8 | acarau
Processing Record 31 of Set 8 | torbay
Processing Record 32 of Set 8 | ouahigouya
Processing Record 33 of Set 8 | nome
Processing Record 34 of Set 8 | guerrero negro
Processing Record 35 of Set 8 | wencheng
Processing Record 36 of Set 8 | san luis
Processing Record 37 of Set 8 | veinticinco de mayo
Processing Record 38 of Set 8 | hamilton
Processing Record 39 of Set 8 | sallins
Processing Record 40 of Set 8 | turukhansk
Processing Record 41 of Set 8 | plastun
Processing Record 42 of Set 8 | haines junction
Processing Record 43 of Set 8 | mandera
Processing Record 44 of Set 8 | asmar
Processing Record 45 of Set 8 | provideniya
Processing Record 46 of Set 8 | mozarlandia
Processing Record 47 of Set 8 | port hedland
Processing Record 48 of Set 8 | progreso
Processing Record 49 of Set 8 | tabas
Processing

Processing Record 11 of Set 12 | porto novo
Processing Record 12 of Set 12 | aliang
Processing Record 13 of Set 12 | atbasar
Processing Record 14 of Set 12 | mildura
Processing Record 15 of Set 12 | kununurra
Processing Record 16 of Set 12 | shenjiamen
Processing Record 17 of Set 12 | gorin
Processing Record 18 of Set 12 | new waterford
Processing Record 19 of Set 12 | balabac
Processing Record 20 of Set 12 | ulaanbaatar
Processing Record 21 of Set 12 | maltahohe
Processing Record 22 of Set 12 | safwah
City not found. Skipping...
Processing Record 23 of Set 12 | nhulunbuy
Processing Record 24 of Set 12 | chicama
Processing Record 25 of Set 12 | mnogovershinnyy
Processing Record 26 of Set 12 | karakendzha
City not found. Skipping...
Processing Record 27 of Set 12 | cervo
Processing Record 28 of Set 12 | vanimo
Processing Record 29 of Set 12 | mingshui
Processing Record 30 of Set 12 | bonga
Processing Record 31 of Set 12 | habra
Processing Record 32 of Set 12 | quatre cocos
Processing Re

Processing Record 47 of Set 15 | hearst
Processing Record 48 of Set 15 | luganville
Processing Record 49 of Set 15 | ucluelet
Processing Record 50 of Set 15 | yalta
Processing Record 1 of Set 16 | cacu
Processing Record 2 of Set 16 | marcona
City not found. Skipping...
Processing Record 3 of Set 16 | menongue
Processing Record 4 of Set 16 | awjilah
Processing Record 5 of Set 16 | alcains
Processing Record 6 of Set 16 | qorveh
Processing Record 7 of Set 16 | matara
Processing Record 8 of Set 16 | anchorage
Processing Record 9 of Set 16 | garango
Processing Record 10 of Set 16 | westerland
Processing Record 11 of Set 16 | lumeje
Processing Record 12 of Set 16 | maxixe
Processing Record 13 of Set 16 | zanjan
Processing Record 14 of Set 16 | sambava
Processing Record 15 of Set 16 | skibbereen
Processing Record 16 of Set 16 | bujaru
Processing Record 17 of Set 16 | manzhouli
Processing Record 18 of Set 16 | cajabamba
Processing Record 19 of Set 16 | sisimiut
Processing Record 20 of Set 16 |

In [7]:
# Print the length of the city_data list to verify how many cities you have
len(city_data)


707

## Add the weather data to a new DataFrame.

In [8]:
# Use the city_data list to create a new pandas DataFrame.
city_data_df = pd.DataFrame(city_data)

In [9]:
# Display sample data
city_data_df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Espanola,35.9911,-106.0806,64.87,8,87,4.00,US,2023-03-06 21:17:20
1,Faya,18.3851,42.4509,62.71,60,23,4.72,SA,2023-03-06 21:17:21
2,Xuanzhou,30.9525,118.7553,55.98,49,97,4.05,CN,2023-03-06 21:17:22
3,San Policarpo,12.1791,125.5072,77.45,83,7,10.92,PH,2023-03-06 21:17:23
4,Puerto Ayora,-0.7393,-90.3518,84.16,88,89,5.01,EC,2023-03-06 21:17:23
5,Rikitea,-23.1203,-134.9692,80.40,78,100,22.84,PF,2023-03-06 21:16:36
6,Fairbanks,64.8378,-147.7164,26.82,69,100,0.00,US,2023-03-06 21:10:20
7,East London,-33.0153,27.9116,70.70,79,83,13.67,ZA,2023-03-06 21:17:25
8,Avarua,-21.2078,-159.7750,87.85,74,40,20.71,CK,2023-03-06 21:17:26
9,Kamenka,51.3223,42.7678,19.63,95,67,6.24,RU,2023-03-06 21:17:27


In [10]:
# Display the DataFrame's column names using the columns Pandas function
city_data_df.columns

Index(['City', 'Lat', 'Lng', 'Max Temp', 'Humidity', 'Cloudiness',
       'Wind Speed', 'Country', 'Date'],
      dtype='object')

In [11]:
# Create a list to reorder the column names as follows:
# "City", "Country", "Lat", "Lng", "Max Temp", "Humidity",  "Cloudiness", "Wind Speed",  "Current Description"
new_column_order = ["City", "Country", "Lat", "Lng", "Max Temp", "Humidity",  "Cloudiness", "Wind Speed",  "Date"]
# Recreate the DataFrame by using the new column order
city_data_df = city_data_df[new_column_order]
# Display sample data
city_data_df

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date
0,Espanola,US,35.9911,-106.0806,64.87,8,87,4.00,2023-03-06 21:17:20
1,Faya,SA,18.3851,42.4509,62.71,60,23,4.72,2023-03-06 21:17:21
2,Xuanzhou,CN,30.9525,118.7553,55.98,49,97,4.05,2023-03-06 21:17:22
3,San Policarpo,PH,12.1791,125.5072,77.45,83,7,10.92,2023-03-06 21:17:23
4,Puerto Ayora,EC,-0.7393,-90.3518,84.16,88,89,5.01,2023-03-06 21:17:23
...,...,...,...,...,...,...,...,...,...
702,Mecca,SA,21.4267,39.8261,82.22,40,10,5.28,2023-03-06 21:36:44
703,Sao Filipe,CV,14.8961,-24.4956,73.00,74,83,5.41,2023-03-06 21:40:23
704,Kamloops,CA,50.6665,-120.3192,36.75,55,100,4.61,2023-03-06 21:39:41
705,Emba,KZ,48.8264,58.1442,36.18,99,100,17.65,2023-03-06 21:40:24


In [12]:
# Display the datatypes of each colunm by accessing the dtypes DataFrame property
city_data_df.dtypes

City           object
Country        object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
Date           object
dtype: object

## Export the DataFrame as a CSV file, and save it as `WeatherPy_Database.csv` in the `Weather_Database` folder.

In [13]:
# Export the city_data DataFrame into a CSV file
city_data_df.to_csv(OUTPUT_DATA_PATH, index_label="City_ID")